In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
#connect to postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.19.jar

--2021-03-08 02:26:03--  https://jdbc.postgresql.org/download/postgresql-42.2.19.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1005078 (982K) [application/java-archive]
Saving to: ‘postgresql-42.2.19.jar’

postgresql-42.2.19. 100%[===================>] 981.52K  --.-KB/s    in 0.1s    

2021-03-08 02:26:04 (6.58 MB/s) - ‘postgresql-42.2.19.jar’ saved [1005078/1005078]



In [3]:
#create Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.19.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://sarahsupercoolbucket.s3.amazonaws.com/combineddata.csv"
spark.sparkContext.addFile(url)
project_df = spark.read.csv(SparkFiles.get("combineddata.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
project_df.show()

+---+--------+--------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+--

In [7]:
#rename index column to county_id
project_df = project_df.withColumnRenamed('_c0', 'county_id')

project_df.show()

+---------+--------+--------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+--------------+----------------------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+-----------+-------------+--------------+-------------+-------------+-----------

In [8]:
#create new dataframe with county and state

county_df = project_df.select(['county_id', 'us_state', 'county'])
county_df.show()

+---------+--------+--------------+
|county_id|us_state|        county|
+---------+--------+--------------+
|        0|      AL|       Baldwin|
|        1|      AL|          Clay|
|        2|      AL|       Colbert|
|        3|      AL|        DeKalb|
|        4|      AL|        Elmore|
|        5|      AL|        Etowah|
|        6|      AL|       Houston|
|        7|      AL|     Jefferson|
|        8|      AL|      Lawrence|
|        9|      AL|       Madison|
|       10|      AL|        Mobile|
|       11|      AL|    Montgomery|
|       12|      AL|        Morgan|
|       13|      AL|       Russell|
|       14|      AL|        Shelby|
|       15|      AL|        Sumter|
|       16|      AL|    Tuscaloosa|
|       17|      AK|Aleutians East|
|       18|      AK|     Anchorage|
|       19|      AK|        Denali|
+---------+--------+--------------+
only showing top 20 rows



In [9]:
#create new df for yearly poverty
poverty_df = project_df.select('county_id', 'poverty_percentage2019', 'poverty_percentage2018', 'poverty_percentage2017', 'poverty_percentage2016')

poverty_df.show()

+---------+----------------------+----------------------+----------------------+----------------------+
|county_id|poverty_percentage2019|poverty_percentage2018|poverty_percentage2017|poverty_percentage2016|
+---------+----------------------+----------------------+----------------------+----------------------+
|        0|                  10.1|                   9.8|                  10.1|                  11.7|
|        1|                  16.6|                  17.6|                  18.1|                  18.9|
|        2|                  14.6|                  13.8|                  15.4|                  16.7|
|        3|                  18.2|                  21.3|                  19.5|                  20.5|
|        4|                  11.4|                  11.2|                 12.01|                  13.5|
|        5|                  18.8|                  17.5|                  18.1|                  17.5|
|        6|                  19.4|                  16.5|       

In [10]:
#create new df for 2019 day breakdown
day_count2019_df = project_df.select('county_id', 'days_with_aqi2019', 'good_days2019', 'moderate_days2019', 'unhealthy_sensitive_days2019', 'unhealthy_days2019', 'very_unhealthy_days2019', 'hazardous_days2019')

day_count2019_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2019|good_days2019|moderate_days2019|unhealthy_sensitive_days2019|unhealthy_days2019|very_unhealthy_days2019|hazardous_days2019|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|              271|          237|               34|                           0|                 0|                      0|                 0|
|        1|              107|           97|               10|                           0|                 0|                      0|                 0|
|        2|              263|          252|               11|                           0|                 0|                      0|                 0|
|        3|              361|          324|               37|                     

In [11]:
#create new df for 2018 day breakdown
day_count2018_df = project_df.select('county_id', 'days_with_aqi2018', 'good_days2018', 'moderate_days2018', 'unhealthy_sensitive_days2018', 'unhealthy_days2018', 'very_unhealthy_days2018', 'hazardous_days2018')

day_count2018_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2018|good_days2018|moderate_days2018|unhealthy_sensitive_days2018|unhealthy_days2018|very_unhealthy_days2018|hazardous_days2018|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            270.0|        245.0|             25.0|                         0.0|               0.0|                    0.0|               0.0|
|        1|            110.0|        103.0|              7.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            277.0|        251.0|             26.0|                         0.0|               0.0|                    0.0|               0.0|
|        3|            350.0|        316.0|             34.0|                     

In [12]:
#create new df for 2017 day breakdown
day_count2017_df = project_df.select('county_id', 'days_with_aqi2017', 'good_days2017', 'moderate_days2017', 'unhealthy_sensitive_days2017', 'unhealthy_days2017', 'very_unhealthy_days2017', 'hazardous_days2017')

day_count2017_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2017|good_days2017|moderate_days2017|unhealthy_sensitive_days2017|unhealthy_days2017|very_unhealthy_days2017|hazardous_days2017|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            270.0|        241.0|             28.0|                         1.0|               0.0|                    0.0|               0.0|
|        1|            118.0|        104.0|             14.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            283.0|        265.0|             18.0|                         0.0|               0.0|                    0.0|               0.0|
|        3|            359.0|        329.0|             30.0|                     

In [13]:
#create new df for 2016 day breakdown
day_count2016_df = project_df.select('county_id', 'days_with_aqi2016', 'good_days2016', 'moderate_days2016', 'unhealthy_sensitive_days2016', 'unhealthy_days2016', 'very_unhealthy_days2016', 'hazardous_days2016')

day_count2016_df.show()

+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|county_id|days_with_aqi2016|good_days2016|moderate_days2016|unhealthy_sensitive_days2016|unhealthy_days2016|very_unhealthy_days2016|hazardous_days2016|
+---------+-----------------+-------------+-----------------+----------------------------+------------------+-----------------------+------------------+
|        0|            279.0|        247.0|             32.0|                         0.0|               0.0|                    0.0|               0.0|
|        1|            116.0|        109.0|              7.0|                         0.0|               0.0|                    0.0|               0.0|
|        2|            282.0|        258.0|             23.0|                         1.0|               0.0|                    0.0|               0.0|
|        3|            348.0|        304.0|             43.0|                     